### Imports and Config

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset, load_metric, Dataset

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

In [2]:
checkpoint = "roberta-base"

### Loading the data

In [3]:
raw_dataset = load_dataset("my_dataset_loading_script", "condemnation")

Reusing dataset condemnation_dataset (/home/geev/.cache/huggingface/datasets/condemnation_dataset/condemnation/1.1.0/ef891eb2986445dbe69883df48d98d7039da591e0c35cb211978c71951a3c83e)


In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1344
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 336
    })
})

In [5]:
raw_train_dataset = raw_dataset["train"]
raw_train_dataset[0]

{'text': 'As multiple sexual allegations come to light with people like <OTHER TARGET 1>, NPR’s -JOHN DOE-, and Michael Fallon, just to name a few...',
 'label': 1}

In [6]:
raw_train_dataset.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['no_condemnation', 'condemnation'], names_file=None, id=None)}

### Preprocessing & Evaluation

In [7]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
def evaluate(trainer, tokenized_dataset):
    predictions = trainer.predict(tokenized_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    def get_metrics(y_pred, y_true):
        metrics ={}
        metrics["accuracy"] = accuracy_score(y_true, y_pred)
        metrics["precision"] = precision_score(y_true, y_pred)
        metrics["recall"] = recall_score(y_true, y_pred)
        metrics["f1"] = f1_score(y_true, y_pred)
        return metrics
    return get_metrics(preds, predictions.label_ids)


### Training with CV

In [9]:
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]
n_split = 10
sss = StratifiedShuffleSplit(n_splits=n_split, test_size=1/n_split, random_state=0)
sss.get_n_splits(train_dataset, train_dataset["label"])

10

In [10]:
torch.cuda.empty_cache()

In [11]:
performance = {"test":{}, "val":{}}
fold = 1
for train_index, val_index in sss.split(train_dataset, train_dataset["label"]):
    cv_train_dataset = Dataset.from_dict(train_dataset[train_index])
    
    training_args = TrainingArguments("test_trainer")
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
    
    trainer = Trainer(
    model,
    training_args,
    train_dataset = cv_train_dataset,
    data_collator = data_collator,
    tokenizer = tokenizer,
    )
    
    trainer.train()
    cv_val_dataset = Dataset.from_dict(train_dataset[val_index])

    performance["val"]["fold "+str(fold)] = evaluate(trainer, cv_val_dataset)
    performance["test"]["fold "+str(fold)] = evaluate(trainer, test_dataset)
    fold+=1
    del model
    del trainer
    del cv_train_dataset
    del cv_val_dataset
    torch.cuda.empty_cache()
    
    

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 135
  Batch size = 16


The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 336
  Batch size = 16
/home/geev/Research/in_progress/Transgressions/transgressions-env/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/geev/.cache/huggingface/transformers/733bade19e5

Step,Training Loss


...



Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 135
  Batch size = 16


The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 336
  Batch size = 16
/home/geev/Research/in_progress/Transgressions/transgressions-env/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/geev/.cache/huggingface/transformers/733bade19e5

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 135
  Batch size = 16


The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 336
  Batch size = 16
/home/geev/Research/in_progress/Transgressions/transgressions-env/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [13]:
import pickle
pickle_dump_file = "10_cv_"+checkpoint+"_performance.p"
pickle.dump(performance, open(pickle_dump_file, "wb"))

In [14]:
pickle.load(open(pickle_dump_file, "rb"))

{'test': {'fold 1': {'accuracy': 0.8422619047619048,
   'precision': 0.8726415094339622,
   'recall': 0.8767772511848341,
   'f1': 0.8747044917257683},
  'fold 2': {'accuracy': 0.7589285714285714,
   'precision': 0.7801724137931034,
   'recall': 0.8578199052132701,
   'f1': 0.8171557562076749},
  'fold 3': {'accuracy': 0.8095238095238095,
   'precision': 0.8127659574468085,
   'recall': 0.9052132701421801,
   'f1': 0.8565022421524663},
  'fold 4': {'accuracy': 0.8065476190476191,
   'precision': 0.8288288288288288,
   'recall': 0.8720379146919431,
   'f1': 0.8498845265588914},
  'fold 5': {'accuracy': 0.7946428571428571,
   'precision': 0.8169642857142857,
   'recall': 0.8672985781990521,
   'f1': 0.8413793103448276},
  'fold 6': {'accuracy': 0.8005952380952381,
   'precision': 0.8185840707964602,
   'recall': 0.8767772511848341,
   'f1': 0.8466819221967964},
  'fold 7': {'accuracy': 0.7946428571428571,
   'precision': 0.8114035087719298,
   'recall': 0.8767772511848341,
   'f1': 0.842

In [15]:
def agg_performance(performance):
    
    agg_performance = {}
    for key in performance:
        agg_performance[key] = {}
        for metric in performance[key]["fold 1"]:
            metric_val_list = [performance[key][fold][metric] for fold in performance[key]]
            agg_performance[key]["avg_"+metric] = "{:.2f}".format(np.mean(metric_val_list))
            agg_performance[key]["std_"+metric] = "{:.2f}".format(np.std(metric_val_list))
    return agg_performance
        
import pprint
pprint.pprint(agg_performance(performance)        )

{'test': {'avg_accuracy': '0.80',
          'avg_f1': '0.84',
          'avg_precision': '0.82',
          'avg_recall': '0.86',
          'std_accuracy': '0.02',
          'std_f1': '0.02',
          'std_precision': '0.02',
          'std_recall': '0.03'},
 'val': {'avg_accuracy': '0.76',
         'avg_f1': '0.82',
         'avg_precision': '0.78',
         'avg_recall': '0.86',
         'std_accuracy': '0.03',
         'std_f1': '0.02',
         'std_precision': '0.03',
         'std_recall': '0.03'}}
